<a href="https://colab.research.google.com/github/mrMichalR/spaceship_titanic/blob/main/spaceship_titanic_PyTorch_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spaceship Titanic project

Welcome to the year 2912, where your data science skills are needed to solve a cosmic mystery. We've received a transmission from four lightyears away and things aren't looking good. The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars. While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension! To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system. Help save them and change history!

In [1]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
# check pytorch version
import torch
print(torch.__version__)

import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from sklearn.linear_model import LogisticRegression
from torch import Tensor
from torch.nn import Linear
# from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

1.13.1+cu116


## Get data 
Get data from link: https://www.kaggle.com/competitions/spaceship-titanic/data and read to your notebook.

In [12]:
path_train = 'https://raw.githubusercontent.com/mrMichalR/spaceship_titanic/main/data/train.csv'
path_test = 'https://raw.githubusercontent.com/mrMichalR/spaceship_titanic/main/data/test.csv'
df_train = pd.read_csv(path_train)#, header=None)
df_test = pd.read_csv(path_test)#, header=None)

df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [13]:
trainloader = DataLoader(df_train, batch_size=13, shuffle=True)
testloader = DataLoader(df_test, batch_size=13, shuffle=True)

### Check if the dataset is balanced (if not think about methods to aquire balance)

In [14]:
# target = "Transported"
print(f'TRAIN target:\n{df_train["Transported"].value_counts()}')

TRAIN target:
True     4378
False    4315
Name: Transported, dtype: int64


Target feature balanced

## PyTorch
Use PyTorch to solve following tasks.

### Implement logistic regression model

In [16]:
# build custom module for logistic regression
class LogisticRegression(torch.nn.Module):    
    # build the constructor
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(13, 1)
    # make predictions
    def forward(self, x):
        x = torch.sigmoid(self.linear(x))
        return x


net = LogisticRegression()

In [17]:
# defining Cross-Entropy loss
criterion = torch.nn.BCELoss()

# defining the optimizer
learning_rate = 0.01
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [ ]:
epochs=20
              
for epoch in tqdm(range(int(epochs)),desc='Training Epochs'): # loop over the dataset multiple times
    running_loss = 0.0

    # for i, data in enumerate(trainloader, 0):
        
        # inputs, labels = data

        # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('\nFinished Training')

In [ ]:
y = df.iloc[:, -1] # -> dtype = bool
X = df.iloc[:, :len(df.columns)-1] # dtype = object + float

# facotorize object features
num_col = X.select_dtypes(exclude=object).columns.to_list()
cat_col = X.select_dtypes(include=object).columns.to_list()
print(type(cat_col))
for col in cat_col:
  X[col] = pd.factorize(X[col])[0]
X = X.astype('float32')

SimpleImputer(strategy='most_frequent', copy=False).fit_transform(X)
MinMaxScaler(copy=False,clip=True).fit_transform(X)
# preprocessor.fit_transform(X)

 
# instantiate the model
input_dim = 13 # Two inputs x1 and x2 
output_dim = 1 # Two possible outputs


# log_regr = make_pipeline(preprocessor, log_regr_model)

log_regr = LogisticRegression(input_dim,output_dim)

 



X_train, X_test, y_train,  y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)


X_train = torch.Tensor(np.array(X_train))
y_train = torch.Tensor(np.array(y_train))

X_test = torch.Tensor(np.array(X_test))
y_test = torch.Tensor(np.array(y_test))


losses = []
losses_test = []
Iterations = []
iter = 0
epochs = 20000
for epoch in tqdm(range(int(epochs)),desc='Training Epochs'):
    # x = X_train
    labels = y_train

    # zero the parameter gradients
    optimizer.zero_grad()

    outputs = log_regr(X_train)
    loss = criterion(torch.squeeze(outputs), labels) # [200,1] -squeeze-> [200]
    loss.backward() # Computes the gradient of the given tensor w.r.t. graph leaves 
    optimizer.step() # Updates weights and biases with the optimizer (SGD)
    
    # print statistics
    running_loss += loss.item()
    
    iter+=1
    if iter%2000==0:
        # calculate Accuracy
        with torch.no_grad():
            # Calculating the loss and accuracy for the test dataset
            correct_test = 0
            total_test = 0
            outputs_test = torch.squeeze(log_regr(X_test))
            loss_test = criterion(outputs_test, y_test)
            
            predicted_test = outputs_test.round().detach().numpy()
            total_test += y_test.size(0)
            correct_test += np.sum(predicted_test == y_test.detach().numpy())
            accuracy_test = 100 * correct_test/total_test
            losses_test.append(loss_test.item())
            
            # Calculating the loss and accuracy for the train dataset
            total = 0
            correct = 0
            total += y_train.size(0)
            correct += np.sum(torch.squeeze(outputs).round().detach().numpy() == y_train.detach().numpy())
            accuracy = 100 * correct/total
            losses.append(loss.item())
            Iterations.append(iter)
            
            print(f"Iteration: {iter}. \nTest - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")
            print(f"Train -  Loss: {loss.item()}. Accuracy: {accuracy}\n")

<class 'list'>
      PassengerId  HomePlanet  CryoSleep   Cabin  Destination   Age  VIP  \
0             0.0         0.0        0.0     0.0          0.0  39.0  0.0   
1             1.0         1.0        0.0     1.0          0.0  24.0  0.0   
2             2.0         0.0        0.0     2.0          0.0  58.0  1.0   
3             3.0         0.0        0.0     2.0          0.0  33.0  0.0   
4             4.0         1.0        0.0     3.0          0.0  16.0  0.0   
...           ...         ...        ...     ...          ...   ...  ...   
8688       8688.0         0.0        0.0  6556.0          2.0  41.0  1.0   
8689       8689.0         1.0        1.0  6557.0          1.0  18.0  0.0   
8690       8690.0         1.0        0.0  6558.0          0.0  26.0  0.0   
8691       8691.0         0.0        0.0  6559.0          2.0  32.0  0.0   
8692       8692.0         0.0        0.0  6559.0          0.0  44.0  0.0   

      RoomService  FoodCourt  ShoppingMall     Spa  VRDeck    Name  
0  

Training Epochs:  11%|█         | 2121/20000 [00:02<00:18, 948.79it/s]

Iteration: 2000. 
Test - Loss: 0.6546239852905273. Accuracy: 72.92944785276073
Train -  Loss: 0.6563275456428528. Accuracy: 72.86770747740346



Training Epochs:  20%|██        | 4023/20000 [00:06<00:49, 320.32it/s]

Iteration: 4000. 
Test - Loss: 0.624670684337616. Accuracy: 72.46932515337423
Train -  Loss: 0.6279553771018982. Accuracy: 72.22678718159409



...

Iteration: 16000. 
Test - Loss: 0.5802053213119507. Accuracy: 72.62269938650307
Train -  Loss: 0.5862243175506592. Accuracy: 72.25965488907148



Training Epochs:  91%|█████████ | 18101/20000 [00:39<00:02, 737.96it/s]

Iteration: 18000. 
Test - Loss: 0.5783540606498718. Accuracy: 72.58435582822086
Train -  Loss: 0.5844627618789673. Accuracy: 72.3582580115037



Training Epochs: 100%|██████████| 20000/20000 [00:41<00:00, 480.13it/s] 

Iteration: 20000. 
Test - Loss: 0.576860785484314. Accuracy: 72.50766871165644
Train -  Loss: 0.5830209255218506. Accuracy: 72.3089564502876




### Use k-fold technique to entry dataset division

In [ ]:
# Test options and evaluation metric
num_folds = 10
# seed = 42
scoring = 'accuracy'

# models=[]
# models.append(('LR', log_regr))

# results = []
# names = []
# kfold = KFold(n_splits=num_folds, shuffle=False, random_state=None)
# cv_results = cross_val_score(log_regr, X_train, y_train, cv=kfold, scoring=scoring)
# results.append(cv_results)
# names.append('LogReg')
# msg = "%s: %f (%f)" % ('LogReg', cv_results.mean(), cv_results.std())
# print(msg)

TypeError: ignored

In [ ]:
# define a cross validation function
def crossvalid(model=None,criterion=None,optimizer=None,dataset=None,k_fold=5):
    
    train_score = pd.Series()
    val_score = pd.Series()
    
    total_size = len(dataset)
    fraction = 1/k_fold
    seg = int(total_size * fraction)
    # tr:train,val:valid; r:right,l:left;  eg: trrr: right index of right side train subset 
    # index: [trll,trlr],[vall,valr],[trrl,trrr]
    for i in range(k_fold):
        trll = 0
        trlr = i * seg
        vall = trlr
        valr = i * seg + seg
        trrl = valr
        trrr = total_size
        # msg
#         print("train indices: [%d,%d),[%d,%d), test indices: [%d,%d)" 
#               % (trll,trlr,trrl,trrr,vall,valr))
        
        train_left_indices = list(range(trll,trlr))
        train_right_indices = list(range(trrl,trrr))
        
        train_indices = train_left_indices + train_right_indices
        val_indices = list(range(vall,valr))
        
        train_set = torch.utils.data.dataset.Subset(dataset,train_indices)
        val_set = torch.utils.data.dataset.Subset(dataset,val_indices)
        
#         print(len(train_set),len(val_set))
#         print()
        train_loader = torch.utils.data.DataLoader(train_set, batch_size=50,
                                          shuffle=True, num_workers=4)
        val_loader = torch.utils.data.DataLoader(val_set, batch_size=50,
                                          shuffle=True, num_workers=4)
        train_acc = train_indices(log_regr,criterion,optimizer,train_loader,epoch=1)
        train_score.at[i] = train_acc
        val_acc = val_set(log_regr,criterion,optimizer,val_loader)
        val_score.at[i] = val_acc
    
    return train_score,val_score

train_score,val_score = crossvalid(log_regr,criterion,optimizer,X_train)


<ipython-input-36-a4b69621af01>:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  train_score = pd.Series()
<ipython-input-36-a4b69621af01>:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  val_score = pd.Series()
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


TypeError: ignored

In [ ]:
in

## Data processing
Prepare data using preprocessing methods.

In [ ]:
target_name = "Transported"
X = df.drop(columns=[target_name])
y = df[target_name]
# y = df[target_name].astype(str)

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
dtypes: float64(6), object(7)
memory usage: 883.0+ KB


### Think and propose how represent categorical values

Proposition:
- impute or drop missing values
- encoding data with e.g. pd.replace 
- scale data

Source: https://www.youtube.com/watch?v=OGpQxIkR4ao

___
Possible simplier solution is to use pandas replace with intiger values - will be checked.
https://www.kaggle.com/code/hussainsheriff/spaceship-titanic-tensor-flow

In [ ]:
for elem in X.columns.to_list():
  if X[elem].nunique() == 2:
    X[elem]=X[elem].astype(float)

In [ ]:
# Ordinal encoder use with encoding unknown values with -1 value

# categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
numerical_preprocessor = StandardScaler()
# numerical_preprocessor = MinMaxScaler()


categorical_columns= X.select_dtypes(include=object).columns.tolist()
numerical_columns = X.select_dtypes(exclude=object).columns.tolist()

In [ ]:
# Defining pipeplines

# categorical_pipeline = Pipeline(
#     steps=[
#         ("Imputer", SimpleImputer(strategy='most_frequent')),
#         ("Encoder", categorical_preprocessor),
#         # ("Imputer", IterativeImputer(random_state=42, skip_complete=True)),
#         # ("Scaler", numerical_preprocessor),
#     ])

numerical_pipeline = Pipeline(
    steps=[
        # ("Imputer", IterativeImputer(random_state=42, skip_complete=True)),
        ("Imputer", SimpleImputer(strategy='most_frequent')),
        ("Scaler", numerical_preprocessor),
    ])

In [ ]:
print(f'Before convert:\n{X.dtypes}\n')

# Convert categorical columns to Pandas category
for col in categorical_columns:
   X[col] = X[col].astype('category')

print(f'After convert:\n{X.dtypes}\n')

Before convert:
PassengerId      object
HomePlanet       object
CryoSleep       float64
Cabin            object
Destination      object
Age             float64
VIP             float64
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
dtype: object

After convert:
PassengerId     category
HomePlanet      category
CryoSleep        float64
Cabin           category
Destination     category
Age              float64
VIP              float64
RoomService      float64
FoodCourt        float64
ShoppingMall     float64
Spa              float64
VRDeck           float64
Name            category
dtype: object



### Think and propose how to handle nulls and nan values

Proposition:
* More pro - impute (e.g. IterativeImputer) - included in above point
* The simplest approach is to remove. In this case dropping caused dataset reduction from 8693 to 6606 -> 2087 rows dropped ! -below


### Divide dataset into train, validation and test subset

In [ ]:
# facotorize object features
num_col = X.select_dtypes(exclude='category').columns.to_list()
cat_col = X.select_dtypes(include='category').columns.to_list()

for col in cat_col:
  X[col] = pd.factorize(X[col])[0]

In [ ]:
X.astype('float32')

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,1.0,0.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,1.0
2,2.0,0.0,0.0,2.0,0.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,2.0
3,3.0,0.0,0.0,2.0,0.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,3.0
4,4.0,1.0,0.0,3.0,0.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,8688.0,0.0,0.0,6556.0,2.0,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,8468.0
8689,8689.0,1.0,1.0,6557.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,8469.0
8690,8690.0,1.0,0.0,6558.0,0.0,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,8470.0
8691,8691.0,0.0,0.0,6559.0,2.0,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,8471.0


Since we have test dataset our train will be split into train and validation.

In [ ]:
#Split of train to train and validation ('val') 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3)

print('TRAIN set:\n', 'X_train:', X_train.shape, '\n y_train2: ', y_train.shape)
print('VAL set:\n', 'X_val:', X_val.shape, '\n y_val2: ', y_val.shape)

TRAIN set:
 X_train: (6085, 13) 
 y_train2:  (6085,)
VAL set:
 X_val: (2608, 13) 
 y_val2:  (2608,)


## Machine learning methods
Use standard ML algorithms to classify passengers.

In [ ]:
# enc_col = categorical_preprocessor.fit_transform(df[cat_col])

preprocessor = ColumnTransformer([
    # ('categorical', categorical_preprocessor, categorical_columns),
    ('numerical', numerical_pipeline, numerical_columns)],  
    #remainder="passthrough"
    )

preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('Imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('Scaler', StandardScaler())]),
                                 ['CryoSleep', 'Age', 'VIP', 'RoomService',
                                  'FoodCourt', 'ShoppingMall', 'Spa',
                                  'VRDeck'])])

### Choose metrics you want to use to evaluate those models,

Accuracy by default but in thi scase recall might be more informative

In [ ]:
# Spot-Check Algorithms
models = []
models.append(('LR', LogisticRegression()))


model_name = []
name = []
for name, model_name in models:
  model = make_pipeline(preprocessor, model_name)
  model.fit(X_train, y_train)
  model.predict(X_val)
  print(f'{name} accuracy: {model.score(X_val, y_val)}')
  print(f'{name} recall: {recall_score(y_val, model.predict(X_val))}')
  # print(f'{name} precision: {precision_score(y_val2, model.predict(X_val2), average="macro")}')
  print(f'{name} f1_score: {f1_score(y_val, model.predict(X_val))}')
  print('\n')

LR accuracy: 0.7756901840490797
LR recall: 0.7678300455235205
LR f1_score: 0.7757761594480643




In [ ]:
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('Imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('Scaler',
                                                                   StandardScaler())]),
                                                  ['CryoSleep', 'Age', 'VIP',
                                                   'RoomService', 'FoodCourt',
                                                   'ShoppingMall', 'Spa',
                                                   'VRDeck'])])),
                ('logisticregression', LogisticRegression())])